Please fill out the information of your group!

| <p style="text-align: center;">First Name</p> | <p style="text-align: center;">Family Name</p> | Matr.-No. |
|-----------------------------------------|-----------------------------------------| --- |
| <p style="text-align: left">*Daniil*</p>     | <p style="text-align: left">*Krechko*</p>     | *k12149099* |
| <p style="text-align: left">*Azat*</p> | <p style="text-align: left">*Vakhitov*</p> | *k12148222* |



<h2 style="text-align: center">344.105/6/7 UE: Natural Language Processing (WS2023/24)</h2>
<h1 style="color:rgb(0,120,170)">Assignment 2</h1>
<h2 style="color:rgb(0,120,170)">Getting to Know Word Embedding!</h2>

<b>Terms of Use</b><br>
This  material is prepared for educational purposes at the Johannes Kepler University (JKU) Linz, and is exclusively provided to the registered students of the mentioned course at JKU. It is strictly forbidden to distribute the current file, the contents of the assignment, and its solution. The use or reproduction of this manuscript is only allowed for educational purposes in non-profit organizations, while in this case, the explicit prior acceptance of the author(s) is required.

**Authors:** Shah Nawaz, Shahed Masoudian<br>


<h2>Table of contents</h2>
<ol>
    <a href="#section-general-guidelines"><li style="font-size:large;font-weight:bold">General Guidelines</li></a>
    <a href="#section-taskA"><li style="font-size:large;font-weight:bold">Task A: Words Similarity and Nearest Neighbors (15 points)</li></a>
    <a href="#section-taskB"><li style="font-size:large;font-weight:bold">Task B: Document Classification with Word Embedding (15 points)</li></a>
    <a href="#section-taskC"><li style="font-size:large;font-weight:bold">Task C: Classification with sent2vec Document Embeddings (2 extra point)</li></a>
    <a href="#section-references"><li style="font-size:large;font-weight:bold">References</li></a>
    
</ol>

<a name="section-general-guidelines"></a><h2 style="color:rgb(0,120,170)">General Guidelines</h2>

### Assignment objective
The aim of this assignment is to get familiarized with using word embedding (WE) models in practice. The assignment in total has **30 points**; it also offers **2 extra points** which can cover any missing point.

This Notebook encompasses all aspects of the assignment, namely the descriptions of tasks as well as your solutions and reports. Feel free to add any required cell for solutions. The cells can contain code, reports, charts, tables, or any other material, required for the assignment. Feel free to provide the solutions in an interactive and visual way! 

Please discuss any unclear point in the assignment in the provided forum in MOODLE. It is also encouraged to provide answers to your peer's questions. However when submitting a post, keep in mind to avoid providing solutions. Please let the tutor(s) know shall you find any error or unclarity in the assignment.


### Libraries & Dataset

The assignment should be implemented with recent versions of `Python` (>3.7). Any standard Python library can be used, so far that the library is free and can be simply installed using `pip` or `conda`. Examples of potentially useful libraries are `scikit-learn`, `numpy`, `scipy`, `gensim`, `nltk`, `spaCy`, and `AllenNLP`. Use the latest stable version of each library.

To conduct the experiments, we use a subset of the `HumSet` dataset [1] (https://blog.thedeep.io/humset/). `HumSet` is created by the DEEP (https://www.thedeep.io) project – an open source platform which aims to facilitate processing of textual data for international humanitarian response organizations. The platform enables the classification of text excerpts, extracted from news and reports into a set of domain specific classes. The provided dataset contains the classes (labels) referring to the humanitarian sectors like agriculture, health, and protection. The dataset contains an overall number of 17,301 data points. 

Download the dataset from the Moodle page of the course.

the provided zip file consists of the following files:
- `thedeep.subset.train.txt`: Train set in csv format with three fields: sentence_id, text, and label.
- `thedeep.subset.validation.txt`: Validation set in csv format with three fields: sentence_id, text, and label.
- `thedeep.subset.test.txt`: Test set in csv format with three fields: sentence_id, text, and label.
- `thedeep.subset.label.txt`: Captions of the labels.
- `thedeep.ToU.txt`: Terms of use of the dataset.

[1] HumSet: Dataset of Multilingual Information Extraction and Classification for Humanitarian Crises Response
*Selim Fekih, Nicolo' Tamagnone, Benjamin Minixhofer, Ranjan Shrestha, Ximena Contla, Ewan Oglethorpe and Navid Rekabsaz.* 
In Findings of the 2022 Conference on Empirical Methods in Natural Language Processing (Findings of EMNLP), December 2022.


## Style 

- Please provide evidence at each state by giving print of the results, provide tables, graphs to further improve the quality of your report


### Submission

Each group should submit the following two files:

- One Jupyter Notebook file (`.ipynb`), containing all the code, results, visualizations, etc. **In the submitted Notebook, all the results and visualizations should already be present, and can be observed simply by loading the Notebook in a browser.** The Notebook must be self-contained, meaning that (if necessary) one can run all the cells from top to bottom without any error. Do not forget to put in your names and student numbers in the first cell of the Notebook. 
- The HTML file (`.html`) achieved from exporting the Jupyter Notebook to HTML (Download As HTML).

You do not need to include the data files in the submission.


<a name="section-taskA"></a><h2 style="color:rgb(0,120,170)">Task A: Words Similarity and Nearest Neighbors (15 points)</h2>


**Loading a word embedding (WE) model (1 points).** Download a pre-trained word embedding model such as word2vec (https://code.google.com/archive/p/word2vec/) or GloVe (https://nlp.stanford.edu/projects/glove/). You can load the downloaded vectors into arrays, or use libraries such as `gensim` to download and process the vectors. 

**Calculating word-to-word similarities (4 points).** Select <ins>5 arbitrary words</ins> from 5 different topics like objects, science disciplines, verbs, adjectives, animals, etc. Let us refer to these words as *source words*. For each source word, calculate its cosine similarities to <ins>6 target words</ins>. The target words of each source word are also selected by you and should cover various levels of semantic relations – according to your linguistic judgement – to the source word, namely from highly-related to not related at all. Organize the target words in tables, such that the target words of each source word are sorted from the highest to the lowest relevance (according to your judgement). Consider the following points:

- **Implementation (2/4 points):** Implement cosine similarity as a function that takes two vectors and returns the similarity score. Implement cosine by yourself and do NOT use the provided functionalities of any library.
- **Reporting and observations (2/4 points):** Report the calculated similarities side by side with your word-to-word semantic relevance judgements in tables. Compare the results and report your observations.  

**Calculating nearest neighbors (10 points).** For the 5 source words, retrieve the $k=10$ nearest neighbors using the word embedding model, namely the words with the highest similarities to the source word. Consider the following points: 
    
- **Overall implementation (3/10 points):** your implemented function takes a source vector, a set of target vectors, and the $k$ parameter, and returns the $k$ nearest neighbors and their similarity scores. Implement nearest neighbor calculation by yourself and do NOT use the provided functionalities of any library.
- **Similarity metrics (2/10 points):** execute the calculation of nearest neighbors according to <ins>two similarity metrics</ins> namely cosine and dot product.
- **Efficiency (3/10 points):** your nearest neighbor functions should provide an *efficient* calculation of nearest neighbors. An inefficient way (which should be avoided!) would be looping over the set of vectors in the word embedding model, and one by one calculating the cosine/dot product similarity of the source vector to each of the target vectors. As a hint for an efficient way, consider that in `numpy` (and other libraries), calculating the dot product of a vector to a matrix is much faster than the dot products of the vector to each vector of the matrix.
- **Reporting and observations (2/10 points):** report the results in tables, which enable comparing between the outputs of the two similarity metrics. Which similarity metric would you prefer? Report your observations.

</div>

In [1]:
# Import necessary libraries
from gensim.models import KeyedVectors

# Load the Word2Vec model
model_path = "/Users/bayve/Desktop/JKU/NLP/word2vec/GoogleNews-vectors-negative300.bin" 
model = KeyedVectors.load_word2vec_format(model_path, binary=False)

# Test the model
#similar_words = model.most_similar('king', topn=5)
#print(similar_words)

In [18]:
import numpy as np
word = model['cat']
np.shape(word)
word1 = model['cup']

In [19]:
#Cosine similarity function
import numpy as np
def cosine_similarity(x, y):  #x, z - vectors
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

#Test
print(cosine_similarity(word, word1))

0.12844744


In [8]:
#Nearest neighbours
def calculate_nearest_neighbours(source_vector, target_vectors, k, cosine: bool, dot: bool):

    # Iterate over all words in the model's vocabulary
    for word in model.index_to_key:  # `model.index_to_key` gives all words
        target_vector = model[word]
        
        if cosine:
            return sorted([(word, cosine_similarity(source_vector, model[word])) for word in model.index_to_key], 
                      key=lambda x: x[1], reverse=True)[:k]
        elif dot:
            return sorted([(word, np.dot(source_vector, model[word])) for word in model.index_to_key], 
                      key=lambda x: x[1], reverse=True)[:k]
        else:
            return "Please choose a similarity metric"
        
        
        
#Test
#choose 5 random words from the model
source_words = ['king', 'apple', 'run', 'blue', 'dog']
target_vectors = [model[word] for word in source_words] 
# Cosine Similarity
print('Cosine Similarity metrics') 
for word, source_vector in zip(source_words, target_vectors):
    nearest_neighbors = calculate_nearest_neighbours(source_vector, model, k=10, cosine=True, dot=False)
    print(f"\nNearest neighbors for '{word}':")
    for similarity, neighbor_word in nearest_neighbors:
        print(f"{neighbor_word}: {similarity}")

# Dot Product Similarity
print('\nDot product similarity metrics')  
for word, source_vector in zip(source_words, target_vectors):
    nearest_neighbors = calculate_nearest_neighbours(source_vector, model, k=10, cosine=False, dot=True)
    print(f"\nNearest neighbors for '{word}':")
    for similarity, neighbor_word in nearest_neighbors:
        print(f"{neighbor_word}: {similarity}")

Cosine Similarity metrics

Nearest neighbors for 'king':
1.0: king
0.7138046026229858: kings
0.6510958075523376: queen
0.6413193941116333: monarch
0.6204220056533813: crown_prince
0.6159994006156921: prince
0.5864823460578918: sultan
0.5797569751739502: ruler
0.5646551847457886: princes
0.5432943105697632: Prince_Paras

Nearest neighbors for 'apple':
1.0: apple
0.7203599214553833: apples
0.6450697779655457: pear
0.641014814376831: fruit
0.6302294135093689: berry
0.6133962273597717: pears
0.605826199054718: strawberry
0.6025873422622681: peach
0.596093475818634: potato
0.5935866236686707: grape

Nearest neighbors for 'run':
1.0000001192092896: run
0.6569936275482178: runs
0.606296718120575: running
0.48340505361557007: drive
0.476498007774353: ran
0.46932128071784973: scamper
0.46402236819267273: tworun_double
0.4631645679473877: go
0.45749351382255554: twoout
0.45697981119155884: walk

Nearest neighbors for 'blue':
1.0000001192092896: blue
0.7225175499916077: red
0.7134225964546204: pu

<a name="section-taskB"></a><h2 style="color:rgb(0,120,170)">Task B: Document Classification with WE (15 points)</h2>


This task follows the same instruction for document classification as provided in Assignment 1. You are indeed free to reuse any part of your code in Assignment 1 for this task. In Assignment 1, the representation of each document was created using a bag of words representation followed by dimensionality reduction. In this task, the document representations are created from the pre-trained word embeddings.

**Map word embeddings to dictionary words (5 points).** For every word in the dictionary (as discussed and created in Assignment 1), fetch the corresponding word embedding from the pre-trained model. If no embedding is found, initialize the corresponding word embedding randomly.

**Document embedding as the average of word embeddings (5 points).** Using the word embeddings, the representation of each document is defined as the *mean of the vectors of each document's words*. In particular, given the document $d$, consisting of words $\left[ v_1, v_2, ..., v_{|d|} \right]$, the document representation $\mathbf{e}_d$ is defined as:

$\mathbf{e}_d = \frac{1}{|d|}\sum_{i=1}^{|d|}{\mathbf{e}_{v_i}}$

where $\mathbf{e}_{v}$ is the vector of the word $v$, and $|d|$ is the length of the document.

**Classification and evaluation (5 points)** Using these new document representations, apply <ins>three classification algorithms</ins> and report the evaluation results (based on accuracy metric) on the test set.



<a name="section-taskC"></a><h2 style="color:rgb(0,120,170)">Task C: Classification with sent2vec Document Embeddings (2 extra point)</h2>


Sent2vec [1] suggests another unsupervised approach to creating document embeddings from the underlying word embeddings. First, using the provided code in the paper, train a sendtvec model on the training set to create document embeddings. Then, repeat Task B while using the document embeddings provided by sent2vec. Similar to Task 2, conduct the classification experiments and report evaluation results.

[1] M. Pagliardini, P. Gupta, and M. Jaggi. Unsupervised Learning of Sentence Embeddings using Compositional n-Gram Features. In Proceedings of the conference of the North American Chapter of the Association for Computational Linguistics (NAACL), 2018.
